# Regression with the Keras Deep Learning

In [2]:

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import math
import numpy as np
import pandas as pd
import pandas

In [3]:
# Import data
nbe_df = pd.read_excel('FL2019 Data.xls', sheet_name='NBE (Coating) 2019')
nbi_df = pd.read_excel('FL2019 Data.xls', sheet_name='NBI 2019')  
env_df = pd.read_excel('FL2019 Data.xls', sheet_name='Environmental Data 2019')
print(nbe_df.head())
print(" ***********************")
print(nbi_df.head())
print(" ***********************")
print(env_df.head())

   STATE STRUCNUM   EN  TOTALQTY  CS1  CS2  CS3  CS4    EPN
0     12   010001  241       366    0  323   43    0    NaN
1     12   010026  241       374  327   34   13    0    NaN
2     12   010028  241       294  220   65    9    0    NaN
3     12   010028  330        23   23    0    0    0    NaN
4     12   010028  515        12   12    0    0    0  330.0
 ***********************
   STATE_CODE_001  STRUCTURE_NUMBER_008  DETOUR_KILOS_019  MAINTENANCE_021  \
0              12                 10001                 0                1   
1              12                 10003                20                2   
2              12                 10006                 3                2   
3              12                 10007                 4                2   
4              12                 10008                16                2   

   Bridge_Age  TRAFFIC_LANES_ON_028A  TRAFFIC_LANES_UND_028B  ADT_029  \
0          47                      6                       0    31500   


In [9]:
nbe_element_df = nbe_df[nbe_df['EN'] == 152] 
print(nbe_element_df)

       STATE STRUCNUM   EN  TOTALQTY    CS1   CS2   CS3  CS4  EPN
2160      12   070033  152       210    194     1    15    0  NaN
2950      12   100100  152       404    351     6    47    0  NaN
5412      12   100618  152       400    400     0     0    0  NaN
6466      12   100920  152       358    346    12     0    0  NaN
6726      12   110077  152       427    421     6     0    0  NaN
6847      12   120001  152        66     59     1     6    0  NaN
7463      12   130006  152       422    223     0   199    0  NaN
8701      12   150112  152       209    163     0    46    0  NaN
9380      12   150225  152       313    302     8     3    0  NaN
10861     12   170054  152       213    141     1    71    0  NaN
11405     12   170167  152       341    341     0     0    0  NaN
11430     12   170168  152       341    341     0     0    0  NaN
11453     12   170169  152       308    258    45     5    0  NaN
11480     12   170170  152       308    246    62     0    0  NaN
13939     

In [4]:
nbe_element_df = nbe_element_df.reset_index(drop=True) 


nbi_df_filter = nbi_df[nbi_df['STRUCTURE_NUMBER_008'].isin(nbe_element_df['STRUCNUM'])]
nbi_df_filter = nbi_df_filter.reindex(index=nbi_df_filter.index[::1]) 
nbi_df_filter = nbi_df_filter.reset_index(drop=True) 

env_df_filter = env_df[env_df['STRUCTURE_'].isin(nbe_element_df['STRUCNUM'])]
env_df_filter = env_df_filter.reindex(index=env_df_filter.index[::1]) 
env_df_filter = env_df_filter.reset_index(drop=True) 

# print(nbe_element_df.head())
# print(nbi_df_filter.head())

# print(nbe_element_df.__len__())
# print(nbi_df_filter.__len__())
print(env_df_filter)

     STATE_CODE  STRUCTURE_  Decimal_La  Decimal_Lo  temp_dp_2m  avg_rh_2m  \
0            12       10065   26.917481  -82.012369   19.507700  81.898201   
1            12       10069   26.890944  -81.998083   19.505800  81.872101   
2            12       10070   26.891147  -81.997856   19.505800  81.872101   
3            12       10073   26.895047  -82.002519   19.505800  81.872101   
4            12       10074   26.895086  -82.002067   19.505800  81.872101   
5            12       10082   26.858139  -81.971333   19.507799  81.854797   
6            12       10083   26.858111  -81.971000   19.507799  81.854797   
7            12       30311   26.171047  -81.731661   19.905600  82.402603   
8            12       30312   26.172814  -81.732758   19.905600  82.402603   
9            12       30313   26.175756  -81.734106   19.905600  82.402603   
10           12       30321   26.272861  -81.742850   19.833599  82.410103   
11           12       30322   26.272889  -81.742556   19.833599 

In [5]:
# Concatenate
# DESIGN_LOAD_031   OPEN_CLOSED_POSTED_041   BRIDGE_CONDITION   SCOUR_CRITICAL_113

#                         nbi_df_filter.DESIGN_LOAD_031,
#                         nbi_df_filter.OPEN_CLOSED_POSTED_041,
#                         nbi_df_filter.SCOUR_CRITICAL_113,
#                         nbi_df_filter.BRIDGE_CONDITION,
env_nbe_nbi_df = pd.concat([
    # NBI data
        nbi_df_filter.MAINTENANCE_021,
        nbi_df_filter.Bridge_Age,
        nbi_df_filter.ADT_029,
        nbi_df_filter.DEGREES_SKEW_034,
        nbi_df_filter.RAILINGS_036A,
        nbi_df_filter.SERVICE_ON_042A,
        nbi_df_filter.SERVICE_UND_042B,
        nbi_df_filter.STRUCTURE_KIND_043A,
        nbi_df_filter.STRUCTURE_TYPE_043B,
        nbi_df_filter.APPR_KIND_044A,
        nbi_df_filter.APPR_TYPE_044B,
        nbi_df_filter.DECK_COND_058,
        nbi_df_filter.SUPERSTRUCTURE_COND_059,
        nbi_df_filter.SUBSTRUCTURE_COND_060,
        nbi_df_filter.CHANNEL_COND_061,
        nbi_df_filter.OPERATING_RATING_064,
        nbi_df_filter.INVENTORY_RATING_066,
        nbi_df_filter.STRUCTURAL_EVAL_067,
        nbi_df_filter.DECK_GEOMETRY_EVAL_068,
        nbi_df_filter.UNDCLRENCE_EVAL_069,
        nbi_df_filter.POSTING_EVAL_070,
        nbi_df_filter.WATERWAY_EVAL_071,
        nbi_df_filter.APPR_ROAD_EVAL_072,
        nbi_df_filter.BRIDGE_IMP_COST_094,
        nbi_df_filter.ROADWAY_IMP_COST_095,
        nbi_df_filter.TOTAL_IMP_COST_096,
        nbi_df_filter.DECK_STRUCTURE_TYPE_107,
        nbi_df_filter.SURFACE_TYPE_108A,
        nbi_df_filter.MEMBRANE_TYPE_108B,
        nbi_df_filter.DECK_PROTECTION_108C,
        nbi_df_filter.PERCENT_ADT_TRUCK_109,
        nbi_df_filter.PIER_PROTECTION_111,
        nbi_df_filter.LOWEST_RATING,

    #Env data
#         env_df_filter.Decimal_La,
#         env_df_filter.Decimal_Lo,
        env_df_filter.temp_dp_2m,
        env_df_filter.avg_rh_2m,
        env_df_filter.temp_a_10m,
        env_df_filter.temp_a_2m,
        env_df_filter.temp_a_60c,
        env_df_filter.temp_s_2cm,
        env_df_filter.ppt_2019,
        env_df_filter.hplus_2019,
        env_df_filter.SO4_2019,
        env_df_filter.Cl_2019,
        env_df_filter.totalN_201,
        env_df_filter.NO3_2019,
        env_df_filter.NH4_2019,
        env_df_filter.Na_2019,
        env_df_filter.K_2019,
        env_df_filter.Mg_2019,
        env_df_filter.Ca_2019,
        # NBE data
        nbe_element_df
        ], 
                       
                       axis=1)

print(env_nbe_nbi_df.head())


   MAINTENANCE_021  Bridge_Age  ADT_029  DEGREES_SKEW_034 RAILINGS_036A  \
0                1          40     4000                13             0   
1                1          40    25750                 0             1   
2                1          40    25750                 0             1   
3                1          40    25750                40             1   
4                1          40    25750                40             1   

   SERVICE_ON_042A  SERVICE_UND_042B  STRUCTURE_KIND_043A  \
0                1                 1                    4   
1                1                 5                    4   
2                1                 5                    4   
3                6                 1                    3   
4                6                 1                    3   

   STRUCTURE_TYPE_043B  APPR_KIND_044A ...  Ca_2019 STATE STRUCNUM   EN  \
0                    2               0 ...  1.35249    12   010065  107   
1                    2          

In [6]:
env_nbe_nbi_df

,MAINTENANCE_021,Bridge_Age,ADT_029,DEGREES_SKEW_034,RAILINGS_036A,SERVICE_ON_042A,SERVICE_UND_042B,STRUCTURE_KIND_043A,STRUCTURE_TYPE_043B,APPR_KIND_044A,...,Ca_2019,STATE,STRUCNUM,EN,TOTALQTY,CS1,CS2,CS3,CS4,EPN
0,1,40,4000,13,0,1,1,4,2,0,...,1.35249,12,010065,107,1305,0,1305,0,0,NaN
1,1,40,25750,0,1,1,5,4,2,0,...,1.35418,12,010069,107,2798,2798,0,0,0,NaN
2,1,40,25750,0,1,1,5,4,2,0,...,1.35418,12,010070,107,2448,2448,0,0,0,NaN
3,1,40,25750,40,1,6,1,3,2,0,...,1.35418,12,010073,107,1663,1267,396,0,0,NaN
4,1,40,25750,40,1,6,1,3,2,0,...,1.35418,12,010074,107,1663,1661,2,0,0,NaN
5,1,42,22750,11,1,6,1,3,2,0,...,1.35627,12,010082,107,1722,1722,0,0,0,NaN
6,1,42,22750,11,1,6,1,3,2,0,...,1.35627,12,010083,107,1722,1722,0,0,0,NaN
7,1,15,1800,0,1,6,5,4,2,0,...,1.49289,12,030311,107,2530,2530,0,0,0,NaN
8,1,15,11000,0,1,6,5,4,2,0,...,1.49289,12,030312,107,912,912,0,0,0,NaN
9,1,15,7600,99,1,6,5,4,2,0,...,1.49289,12,030313,107,2128,2128,0,0,0,NaN


In [7]:
#         "DESIGN_LOAD_031",
#         "OPEN_CLOSED_POSTED_041",
#         "SCOUR_CRITICAL_113",
#         "BRIDGE_CONDITION",
cols = ["MAINTENANCE_021",
        "Bridge_Age",
        "ADT_029",
        "DEGREES_SKEW_034",
        "RAILINGS_036A",
        "SERVICE_ON_042A",
        "SERVICE_UND_042B",
        "STRUCTURE_KIND_043A",
        "STRUCTURE_TYPE_043B",
        "APPR_KIND_044A",
        "APPR_TYPE_044B",
        "DECK_COND_058",
        "SUPERSTRUCTURE_COND_059",
        "SUBSTRUCTURE_COND_060",
        "CHANNEL_COND_061",
        "OPERATING_RATING_064",
        "INVENTORY_RATING_066",
        "STRUCTURAL_EVAL_067",
        "DECK_GEOMETRY_EVAL_068",
        "UNDCLRENCE_EVAL_069",
        "POSTING_EVAL_070",
        "WATERWAY_EVAL_071",
        "APPR_ROAD_EVAL_072",
        "BRIDGE_IMP_COST_094",
        "ROADWAY_IMP_COST_095",
        "TOTAL_IMP_COST_096",
        "DECK_STRUCTURE_TYPE_107",
        "SURFACE_TYPE_108A",
        "MEMBRANE_TYPE_108B",
        "DECK_PROTECTION_108C",
        "PERCENT_ADT_TRUCK_109",
        "PIER_PROTECTION_111",
        "LOWEST_RATING",
        "EPN"
       ]

env_nbe_nbi_df[cols] = env_nbe_nbi_df[cols].replace({np.nan:0,'N':0})


In [8]:
env_nbe_nbi_df

,MAINTENANCE_021,Bridge_Age,ADT_029,DEGREES_SKEW_034,RAILINGS_036A,SERVICE_ON_042A,SERVICE_UND_042B,STRUCTURE_KIND_043A,STRUCTURE_TYPE_043B,APPR_KIND_044A,...,Ca_2019,STATE,STRUCNUM,EN,TOTALQTY,CS1,CS2,CS3,CS4,EPN
0,1,40,4000,13,0,1,1,4,2,0,...,1.35249,12,010065,107,1305,0,1305,0,0,0.0
1,1,40,25750,0,1,1,5,4,2,0,...,1.35418,12,010069,107,2798,2798,0,0,0,0.0
2,1,40,25750,0,1,1,5,4,2,0,...,1.35418,12,010070,107,2448,2448,0,0,0,0.0
3,1,40,25750,40,1,6,1,3,2,0,...,1.35418,12,010073,107,1663,1267,396,0,0,0.0
4,1,40,25750,40,1,6,1,3,2,0,...,1.35418,12,010074,107,1663,1661,2,0,0,0.0
5,1,42,22750,11,1,6,1,3,2,0,...,1.35627,12,010082,107,1722,1722,0,0,0,0.0
6,1,42,22750,11,1,6,1,3,2,0,...,1.35627,12,010083,107,1722,1722,0,0,0,0.0
7,1,15,1800,0,1,6,5,4,2,0,...,1.49289,12,030311,107,2530,2530,0,0,0,0.0
8,1,15,11000,0,1,6,5,4,2,0,...,1.49289,12,030312,107,912,912,0,0,0,0.0
9,1,15,7600,99,1,6,5,4,2,0,...,1.49289,12,030313,107,2128,2128,0,0,0,0.0


In [9]:
env_nbe_nbi_df.columns

Index(['MAINTENANCE_021', 'Bridge_Age', 'ADT_029', 'DEGREES_SKEW_034',
       'RAILINGS_036A', 'SERVICE_ON_042A', 'SERVICE_UND_042B',
       'STRUCTURE_KIND_043A', 'STRUCTURE_TYPE_043B', 'APPR_KIND_044A',
       'APPR_TYPE_044B', 'DECK_COND_058', 'SUPERSTRUCTURE_COND_059',
       'SUBSTRUCTURE_COND_060', 'CHANNEL_COND_061', 'OPERATING_RATING_064',
       'INVENTORY_RATING_066', 'STRUCTURAL_EVAL_067', 'DECK_GEOMETRY_EVAL_068',
       'UNDCLRENCE_EVAL_069', 'POSTING_EVAL_070', 'WATERWAY_EVAL_071',
       'APPR_ROAD_EVAL_072', 'BRIDGE_IMP_COST_094', 'ROADWAY_IMP_COST_095',
       'TOTAL_IMP_COST_096', 'DECK_STRUCTURE_TYPE_107', 'SURFACE_TYPE_108A',
       'MEMBRANE_TYPE_108B', 'DECK_PROTECTION_108C', 'PERCENT_ADT_TRUCK_109',
       'PIER_PROTECTION_111', 'LOWEST_RATING', 'temp_dp_2m', 'avg_rh_2m',
       'temp_a_10m', 'temp_a_2m', 'temp_a_60c', 'temp_s_2cm', 'ppt_2019',
       'hplus_2019', 'SO4_2019', 'Cl_2019', 'totalN_201', 'NO3_2019',
       'NH4_2019', 'Na_2019', 'K_2019', 'Mg_2019',

# Normalized 

In [10]:
from sklearn.preprocessing import MinMaxScaler
ss = MinMaxScaler()
scale_features = ['MAINTENANCE_021', 'Bridge_Age', 'ADT_029', 'DEGREES_SKEW_034',
       'RAILINGS_036A', 'SERVICE_ON_042A', 'SERVICE_UND_042B',
       'STRUCTURE_KIND_043A', 'STRUCTURE_TYPE_043B', 'APPR_KIND_044A',
       'APPR_TYPE_044B', 'DECK_COND_058', 'SUPERSTRUCTURE_COND_059',
       'SUBSTRUCTURE_COND_060', 'CHANNEL_COND_061', 'OPERATING_RATING_064',
       'INVENTORY_RATING_066', 'STRUCTURAL_EVAL_067', 'DECK_GEOMETRY_EVAL_068',
       'UNDCLRENCE_EVAL_069', 'POSTING_EVAL_070', 'WATERWAY_EVAL_071',
       'APPR_ROAD_EVAL_072', 'BRIDGE_IMP_COST_094', 'ROADWAY_IMP_COST_095',
       'TOTAL_IMP_COST_096', 'DECK_STRUCTURE_TYPE_107', 'SURFACE_TYPE_108A',
       'MEMBRANE_TYPE_108B', 'DECK_PROTECTION_108C', 'PERCENT_ADT_TRUCK_109',
       'PIER_PROTECTION_111', 'LOWEST_RATING', 'temp_dp_2m', 'avg_rh_2m',
       'temp_a_10m', 'temp_a_2m', 'temp_a_60c', 'temp_s_2cm', 'ppt_2019',
       'hplus_2019', 'SO4_2019', 'Cl_2019', 'totalN_201', 'NO3_2019',
       'NH4_2019', 'Na_2019', 'K_2019', 'Mg_2019', 'Ca_2019', 'STATE',
       'STRUCNUM', 'EN', 'TOTALQTY', 'CS1', 'CS2', 'CS3', 'CS4', 'EPN']
env_nbe_nbi_df[scale_features] = ss.fit_transform(env_nbe_nbi_df[scale_features])

In [11]:
env_nbe_nbi_df

,MAINTENANCE_021,Bridge_Age,ADT_029,DEGREES_SKEW_034,RAILINGS_036A,SERVICE_ON_042A,SERVICE_UND_042B,STRUCTURE_KIND_043A,STRUCTURE_TYPE_043B,APPR_KIND_044A,...,Ca_2019,STATE,STRUCNUM,EN,TOTALQTY,CS1,CS2,CS3,CS4,EPN
0,0.000000,0.352381,0.014944,0.131313,0.0,0.000000,0.111111,0.75,0.066667,0.000000,...,0.585873,0.0,0.000000,0.0,0.038619,0.000000,0.157647,0.000000,0.0,0.0
1,0.000000,0.352381,0.106504,0.000000,1.0,0.000000,0.555556,0.75,0.066667,0.000000,...,0.587343,0.0,0.000004,0.0,0.089870,0.095986,0.000000,0.000000,0.0,0.0
2,0.000000,0.352381,0.106504,0.000000,1.0,0.000000,0.555556,0.75,0.066667,0.000000,...,0.587343,0.0,0.000005,0.0,0.077855,0.083979,0.000000,0.000000,0.0,0.0
3,0.000000,0.352381,0.106504,0.404040,1.0,0.714286,0.111111,0.50,0.066667,0.000000,...,0.587343,0.0,0.000009,0.0,0.050908,0.043465,0.047838,0.000000,0.0,0.0
4,0.000000,0.352381,0.106504,0.404040,1.0,0.714286,0.111111,0.50,0.066667,0.000000,...,0.587343,0.0,0.000010,0.0,0.050908,0.056981,0.000242,0.000000,0.0,0.0
5,0.000000,0.371429,0.093875,0.111111,1.0,0.714286,0.111111,0.50,0.066667,0.000000,...,0.589162,0.0,0.000018,0.0,0.052933,0.059074,0.000000,0.000000,0.0,0.0
6,0.000000,0.371429,0.093875,0.111111,1.0,0.714286,0.111111,0.50,0.066667,0.000000,...,0.589162,0.0,0.000019,0.0,0.052933,0.059074,0.000000,0.000000,0.0,0.0
7,0.000000,0.114286,0.005683,0.000000,1.0,0.714286,0.555556,0.75,0.066667,0.000000,...,0.708015,0.0,0.021749,0.0,0.080670,0.086792,0.000000,0.000000,0.0,0.0
8,0.000000,0.114286,0.044412,0.000000,1.0,0.714286,0.555556,0.75,0.066667,0.000000,...,0.708015,0.0,0.021750,0.0,0.025128,0.031286,0.000000,0.000000,0.0,0.0
9,0.000000,0.114286,0.030099,1.000000,1.0,0.714286,0.555556,0.75,0.066667,0.000000,...,0.708015,0.0,0.021751,0.0,0.066870,0.073002,0.000000,0.000000,0.0,0.0


In [12]:
env_nbe_nbi_df.columns

Index(['MAINTENANCE_021', 'Bridge_Age', 'ADT_029', 'DEGREES_SKEW_034',
       'RAILINGS_036A', 'SERVICE_ON_042A', 'SERVICE_UND_042B',
       'STRUCTURE_KIND_043A', 'STRUCTURE_TYPE_043B', 'APPR_KIND_044A',
       'APPR_TYPE_044B', 'DECK_COND_058', 'SUPERSTRUCTURE_COND_059',
       'SUBSTRUCTURE_COND_060', 'CHANNEL_COND_061', 'OPERATING_RATING_064',
       'INVENTORY_RATING_066', 'STRUCTURAL_EVAL_067', 'DECK_GEOMETRY_EVAL_068',
       'UNDCLRENCE_EVAL_069', 'POSTING_EVAL_070', 'WATERWAY_EVAL_071',
       'APPR_ROAD_EVAL_072', 'BRIDGE_IMP_COST_094', 'ROADWAY_IMP_COST_095',
       'TOTAL_IMP_COST_096', 'DECK_STRUCTURE_TYPE_107', 'SURFACE_TYPE_108A',
       'MEMBRANE_TYPE_108B', 'DECK_PROTECTION_108C', 'PERCENT_ADT_TRUCK_109',
       'PIER_PROTECTION_111', 'LOWEST_RATING', 'temp_dp_2m', 'avg_rh_2m',
       'temp_a_10m', 'temp_a_2m', 'temp_a_60c', 'temp_s_2cm', 'ppt_2019',
       'hplus_2019', 'SO4_2019', 'Cl_2019', 'totalN_201', 'NO3_2019',
       'NH4_2019', 'Na_2019', 'K_2019', 'Mg_2019',

In [13]:
# total 34
# total 59
parameter_input_number = 4
parameter_output_number = 4

# dataset = env_nbe_nbi_df.values
# # split into input (X) and output (Y) variables
# X = dataset[:,0:parameter_input_number].astype('float32')
# Y = dataset[:,-5:-1].astype('float32')

X = env_nbe_nbi_df[['NH4_2019','K_2019','temp_dp_2m','NO3_2019']]
Y = env_nbe_nbi_df[['CS1','CS2','CS3','CS4']]

# X_train, X_test, y_train, y_test = np.asarray(train_test_split(X, Y, test_size=0.1))

In [14]:
X

,NH4_2019,K_2019,temp_dp_2m
0,0.364448,0.075804,0.718503
1,0.368577,0.084824,0.718141
2,0.368577,0.084824,0.718141
3,0.368577,0.084824,0.718141
4,0.368577,0.084824,0.718141
5,0.373237,0.094585,0.718522
6,0.373237,0.094585,0.718522
7,0.580814,0.428065,0.794492
8,0.580814,0.428065,0.794492
9,0.580814,0.428065,0.794492


In [15]:
Y

,CS1,CS2
0,0.000000,0.157647
1,0.095986,0.000000
2,0.083979,0.000000
3,0.043465,0.047838
4,0.056981,0.000242
5,0.059074,0.000000
6,0.059074,0.000000
7,0.086792,0.000000
8,0.031286,0.000000
9,0.073002,0.000000


## Baseline Neural Network Model

In [16]:
# define base model
# 34 inputs  -> 4 output
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(parameter_input_number, input_dim=parameter_input_number, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [17]:
# evaluate model
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Baseline: %.8f (%.8f) MSE" % (results.mean(), results.std()))


Baseline: -0.00684226 (0.00634357) MSE


## Modeling The Standardized Dataset

In [18]:

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.8f (%.8f) MSE" % (results.mean(), results.std()))

Standardized: -0.00696483 (0.00626285) MSE


## Evaluate a Deeper Network Topology

In [19]:

# define the model

# 34 inputs -> [34 -> 34/2 ] -> 4 output
def larger_model():
	# create model
	model = Sequential()
	model.add(Dense(parameter_input_number, input_dim=parameter_input_number, kernel_initializer='normal', activation='relu'))
	model.add(Dense(parameter_input_number/2, kernel_initializer='normal', activation='relu'))
	model.add(Dense(parameter_output_number, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [20]:

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.8f (%.8f) MSE" % (results.mean(), results.std()))

Larger: -0.00594178 (0.00653397) MSE


##  Change Network Dense 

In [21]:
# define the model

# 34*2 inputs -> [34*2 -> 34 ]  ->34 middle  -> 4 output
def larger_model():
	# create model
	model = Sequential()
	model.add(Dense(parameter_input_number*2, input_dim=parameter_input_number, kernel_initializer='normal', activation='relu'))
	model.add(Dense(parameter_input_number, kernel_initializer='normal', activation='relu'))
	model.add(Dense(parameter_output_number, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [22]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.8f (%.8f) MSE" % (results.mean(), results.std()))

Larger: -0.00619117 (0.00651922) MSE
